# Standard libraries

Like pandapower, also pandapipes provides different standard libraries to simplify the workflow. In comparison with pandapower, pandapipes does not only provide a component library, but also a fluid library. After the pandapipes installation, this library already contains several fluids. Of course, it is also possible to define new fluids and add them to the library.

This tutorial demonstrates the usage with the libraries and shows, how components and fluids can be loaded and saved. We start with a tutorial on the component library. This library contains pipes and pumps.


## pandapipes component library

With the function "get_data", which is defined in the module std_type_toolbox, all entries in the library can be displayed. Necessary parameters include the path to the library file and the type of the component. In the following section, the pipe entries are plotted. 

In [ ]:
from pandapipes.std_types.std_type_class import get_data
print(get_data(r'../pandapipes/std_types/library/Pipe.csv', 'pipe'))

The csv files can be extended. We will now create an example network using components from the library.

In [ ]:
import pandapipes as pp
net = pp.create_empty_network()
junction1 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Connection to External Grid")
junction2 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction")
junction3 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction to sink")
medium_pressure_grid = pp.create_ext_grid(net, junction=junction1, fluid="water", p_bar=1, t_k=293.15, name="Grid Connection")
sink = pp.create_sink(net, junction=junction3, mdot_kg_per_s=23.05, name="Sink")

In the code above, a pipe connection is missing between junction1 and junction2. We load the pipe type "300_GGG" from the library using the function "create_pipe". Compared to the function "create_pipe_from_parameters" a parameter with a library reference can be specified.

In [ ]:
pipe1 = pp.create_pipe(net, from_junction=junction2, to_junction=junction3, length_km=1, std_type="300_GGG", name="Pipe 1")

Between the external grid and the pipe, there is also a connection missing. This one will be defined using a pump component. The pressure difference is defined based on the underlying pump curve.

In [ ]:
P1 = pp.create_pump(net, from_junction=junction1, to_junction=junction2, std_type="P1")

The network can be computed as usual.

In [ ]:
pp.pipeflow(net)
net.res_pump

The pump raises the pressure to 4 bar. We can check this by having a look at the pump curve.

In [ ]:
print(get_data(r'../pandapipes/std_types/library/Pump/P1.csv', 'pump'))

We notice that the pressure rais of 4 bar corresponds to a volume flow of 83 m^3/h. This volume flow is also defined at the sink.

## The fluid library

In the introductory example, the fluid is added via a parameter of the create_empty_network function. Internally, pandapipes fetches the corresponding fluid properties, which are saved for the specified fluid. Some of the predefined fluids are

- air
- hydrogen
- lgas
- hgas
- water
- methane

For the calculation of pressure and flow velocity the density and viscosity has to be known. In case of gas calculations, these fluid properties can be dependend on temperature. If the temperature is calculated, further material properties may be needed, for example the heat capacity of the fluid.

An overview of defined fluid properties is given in the directory pandapipes\properties. In the directory of each fluid, you can find the fluid properties. Each property is described by a text file. This file contains data points. During the calculation, pandapipes interpolates between these points.

If you want to know which properties are defined in the current net container, you can use the function get_property().

In [ ]:
net.fluid.get_property("density", 295)

The second parameter is the temperature, at which the density is displayed.

## Defining new fluids

New fluids can be created or existing fluids modified. The mentioned fluids so far were dependent on temperature. We will now create an incompressible fluid with constant material properties and add it to the net.

In a first step, the material properties are created. These properties are then added to a fluid container. The data structures used to describe properties and fluids are pythin classes.

In [ ]:
import pandapipes.properties.fluids as fl
density_const = fl.FluidPropertyConstant(900)
viscosity_const = fl.FluidPropertyConstant(0.002)
heat_capacity_const = fl.FluidPropertyConstant(4000)

The three properties density, viscosity and heat capacity have to be defined. For every property, an object of the class FluidPropertyConstant is created and the constant property value is used in the object constructor. All properties have to be entered using SI units.

Although constant properties are added here, pandapipes also provides classes for inter- and extrapolating properties.

Afterwards, a fluid container is created and the properties are added.

In [ ]:
water_const =  fl.Fluid("water_const","liquid")
water_const.add_property(property_name="density",prop=density_const)
water_const.add_property(property_name="viscosity",prop=viscosity_const)
water_const.add_property(property_name="heat_capacity",prop= heat_capacity_const)

The fluid can now be used to calculate incompressible media. With the command ...:

In [ ]:
fl._add_fluid_to_net(net,water_const)

... the fluid is added to the net container. By plotting again the fluid properties, we see that the old fluid has been replaced. 

In [ ]:
net.fluid